In [ ]:
import cv2
import numpy as np
from PIL import Image
import torch
from torchvision import transforms
import matplotlib.pyplot as plt
from easydict import EasyDict as edict
import parser
import os
import network
import warnings
import torch.nn.functional as F
warnings.filterwarnings('ignore')
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [ ]:
t = transforms.Compose([transforms.Resize((224, 224)), 
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
args = edict({'brightness': None,
 'cache_refresh_rate': 1000,
 'contrast': None,
 'criterion': 'triplet',
 'dataset_name': 'st_lucia',
 'datasets_folder': './datasets',
 'dense_feature_map_size': [61, 61, 128],
 'device': 'cuda',
 'efficient_ram_testing': False,
 'epochs_num': 50,
 'features_dim': 1024,
 'foundation_model_path': None,
 'horizontal_flip': False,
 'hue': None,
 'infer_batch_size': 16,
 'l2': 'before_pool',
 'lr': 1e-05,
 'majority_weight': 0.01,
 'margin': 0.1,
 'mining': 'partial',
 'neg_samples_num': 1000,
 'negs_num_per_query': 2,
 'num_workers': 8,
 'optim': 'adam',
 'patience': 3,
 'pca_dataset_folder': None,
 'pca_dim': None,
 'queries_per_epoch': 5000,
 'rand_perspective': None,
 'random_resized_crop': None,
 'random_rotation': None,
 'recall_values': [1, 5, 10, 20],
 'registers': False,
 'rerank_num': 100,
 'resize': [224, 224],
 'resume': 'ckpts/SelaVPR_msls.pth',
 'saturation': None,
 'save_dir': 'test/default/2025-02-06_13-22-01',
 'seed': 0,
 'test_method': 'hard_resize',
 'train_batch_size': 4,
 'train_positives_dist_threshold': 10,
 'val_positive_dist_threshold': 25})
model = network.GeoLocalizationNet(args)
model = model.to(args.device)
model = torch.nn.DataParallel(model)
state_dict = torch.load(args.resume)["model_state_dict"]
model.load_state_dict(state_dict)

In [ ]:
imgpath = "/robodata/smodak/VPR/SelaVPR/datasets/san_francisco/images/test/database/@0550313.87@4184192.09@10@S@037.80373@-122.42845@14666@00@089@003@@@@@.jpg"
img = Image.open(imgpath).convert('RGB')
H, W = np.asarray(img).shape[:2]  # Original image dimensions
img = t(img).unsqueeze(0).to(args.device)
with torch.inference_mode():
    fm = model.module.backbone(img)["x_norm_patchtokens"]
fm2 = fm.squeeze().reshape(16, 16, 1024)
A = fm2.mean(dim=-1)
A_resized = F.interpolate(A.unsqueeze(0).unsqueeze(0), size=(H, W), mode='bilinear', align_corners=False)
A_resized = A_resized.squeeze().cpu().detach().numpy()
A_resized = np.abs(A_resized)

In [ ]:
def visualize_attention_map(image, attention_map):
    fig, ax = plt.subplots()
    ax.imshow(image)  # Display original image
    im = ax.imshow(attention_map, cmap='jet', alpha=0.5)  # Overlay heatmap with transparency
    ax.axis('off')
    # Add colorbar
    cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.ax.tick_params(labelsize=8)  # Adjust colorbar label size for readability
    plt.show()
visualize_attention_map(Image.open(imgpath), A_resized)

In [ ]:
from match_image import *
import os
sela = SelaVPRminimal()
img_dir = "bags/route"
img_list = sorted(os.listdir(img_dir))
img_paths = [os.path.join(img_dir, img) for img in img_list]

In [ ]:
query_index = 19
candidate_indices = [3, 4, 5, 6, 7, 10, 20, 100]

In [ ]:
query_imgpath = img_paths[query_index]
candidate_imgpaths = [img_paths[i] for i in candidate_indices]

In [ ]:
indices = sela.match_image(query_imgpath, candidate_imgpaths)
print([candidate_indices[i] for i in indices.squeeze().tolist()])